In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import sparse
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from time import perf_counter
# For multiprocessing
import multiprocessing
from psutil import cpu_count
# This one, with logical=False, is better than multiprocessing.cpu_count
# https://stackoverflow.com/questions/40217873/multiprocessing-use-only-the-physical-cores

# Change the forking method used by multiprocessing, avoids errors on Mac OS Catalina
# e.g. https://github.com/matplotlib/matplotlib/issues/15410
multiprocessing.set_start_method('forkserver')

In [ ]:
plt.rcParams["figure.figsize"] = (4, 3)

# Functions related to model for odorants and receptors
From Reddy et al. 2018, the way to define an odorant is to specify a vector of binding affinities, $\vec{\kappa}$, and a vector of activation efficacies, $\vec{\eta}$. For a background, we define $n_{mix}$ such odorants and combine them into $\vec{\eta}_{mix}$ and $\vec{\kappa}_{mix}$. 

In [ ]:
def generate_odorant(n_rec, rgen, lambda_in=0.1):
    """ Generate vectors eta and kappa^-1 for an odorant, with antagonism parameter rho. 
    
    Args:
        n_rec (int): number of receptor types, length of vectors
        mean_in (float): average value of the input vector. 
        rgen (np.random.Generator): random generate (numpy >= 1.17)
    Returns:
        kappa1_vec (np.ndarray): 1d vector of receptor activities
    """
    return rgen.exponential(scale=1.0/lambda_in, size=n_rec)

def generate_background(n_rec, rgen, lambda_in=10.0):
    """ Generate vectors eta and kappa^-1 for an odorant, with antagonism parameter rho. 
    
    Args:
        n_rec (int): number of receptor types, length of vectors
        mean_in (float): average value of the input vector. 
        rgen (np.random.Generator): random generate (numpy >= 1.17)
    Returns:
        kappa1_vec (np.ndarray): 1d vector of receptor activities
    """
    raise NotImplementedError("You should use generate_odorant instead")

### Tests

In [ ]:
# Tests
inputs_test = []
for i in range(100):
    odi = generate_odorant(50, np.random.default_rng(seed=73924+i))
    inputs_test.append(odi)
inputs_test = np.concatenate(inputs_test).flatten()

In [ ]:
# The following histograms should look gaussian with standard deviations 1 and 4
fig, ax = plt.subplots()
fig.set_size_inches(2.5, 2.5)
ax.hist(inputs_test)
ax.set_xlabel(r"Receptor input")
fig.tight_layout()
plt.show()
plt.close()

# Functions related to the olfactory network model

In [ ]:
def project_neural_tag(s_vec, w_vec, projmat, kc_sparsity=0.05, adapt_kc=True, n_pn_per_kc=3, fix_thresh=None):
    """ Given the parameters of the Shen 2020 neural network, project the input layer s_vec
    with the inhibitory feedback weights w_vec to the sparse kenyon cell (KC) output, 
    thresholding KCs below kc_thresh and then keeping only a fraction kc_sparsity of active KCs.
    
    Args:
        s_vec (np.ndarray): input vector, activation of each receptor type
        w_vec (np.ndarray): inhibition weights from LN1 to PN neurons
        projmat (np.ndarray or sp.sparse.csr_matrix): projection matrix from PNs to KCs, 
            shape n_kc x n_receptors. Will use the .dot method of the matrix. 

    Returns:
        z_set (set): sparse neural tag for the given activation odor. List of active KCs. 
    """
    # Extract useful information
    n_kc = projmat.shape[0]
    n_rec = projmat.shape[1]
    # Number of connections from PNs to each KC. If three, then threshold equals mean of inputs
    # Otherwise we need to correct for the fact that the more KCs project to each PN, 
    # the higher the signal of each PN will be compared to the case described in the paper (3 KCs to each PN)
    if fix_thresh is not None:
        kc_thresh = fix_thresh
    elif adapt_kc:
        kc_thresh = np.mean(s_vec) * n_pn_per_kc / 3
    else:
        kc_thresh = np.mean(s_vec)

    # 1. Project on PNs, including inhibition from LN1
    x_vec = np.maximum(s_vec - w_vec, 0)
    #x_vec = s_vec - w_vec
    #x_vec[x_vec<0] = 0
    
    # 2. Project on KCs
    y_vec = projmat.dot(x_vec)
    
    # 3. Threshold noise:: will consider only positions in mask. 
    mask = (y_vec >= kc_thresh).astype(bool)
    y_vec[np.logical_not(mask)] = 0.0
    #mask2 = (projmat.dot(s_vec/6) >= kc_thresh)
    #print("Non-zero elements after thresholding by mean {}: ".format(kc_thresh), np.count_nonzero(mask))
    #print("Compare to expected:", np.count_nonzero(mask2))
    
    # 4. Binarize: keep the np.ceil(0.05*n_kc) most active KCs non-zero KCs, 
    # or all the nonzero ones. Return a set of the indices of those cells. 
    # No arbitrary tie breaks, so can't just sort and take the first 0.05n_kc args
    # TODO: make sure this is a fine format and we don't need the full vector
    # Otherwise, use an array of booleans for the binary vector, or a sp.sparse?
    thresh_keep = np.quantile(y_vec, 1.0 - kc_sparsity)
    z_set = set(np.nonzero(np.logical_and(mask, y_vec >= thresh_keep))[0])
    #y_vec[y_vec < thresh_keep] = 0.0
    #z_set = set(np.nonzero(y_vec)[0])
    return z_set

def jaccard(s1, s2):
    if len(s1) > 0 or len(s2) > 0:  # includes the special case of s1 and s2 empty sets
        return len(s1 & s2) / len(s1 | s2)
    else:
        return 0.0

In [ ]:
def create_sparse_proj_mat(n_kc, n_rec, rgen, fraction_filled=6/50):
    n_per_row = int(fraction_filled*n_rec)
    data = np.ones(n_per_row*n_kc, dtype="uint8")
    row_ind = np.arange(n_per_row*n_kc, dtype=int) // n_per_row
    col_ind = np.ones((n_kc, n_per_row), dtype="int")
    for i in range(n_kc):
        col_ind[i] = rgen.choice(n_rec, size=n_per_row, replace=False)
    mat = sp.sparse.csr_matrix((data, (row_ind, col_ind.ravel())), shape=(n_kc, n_rec), dtype="uint8")
    return mat

### Tests

In [ ]:
# Tests
# Building a random matrix for testing
rdgen_test = np.random.default_rng(8438)
proj_mat = create_sparse_proj_mat(2000, 50, rdgen_test)

odor_test = generate_odorant(50, rdgen_test)
# Low concentration regime, where very few KC get activated at all
tag_test = project_neural_tag(odor_test, np.zeros(50), proj_mat)
print(len(tag_test))
print(tag_test)

In [ ]:
# check locality preservation: generate a bunch of odorants, compute their tags, 
# and compare the similarity of their tags based on the similariy of the s_vecs 
# (or the eta vecs and kappa_vecs?)
# This isn't the best test, but any positive correlation is a good sign. 
n_test = 1000
tag_test = project_neural_tag(odor_test, np.zeros(50), proj_mat)
tags_distances = np.zeros(n_test)
inputs_distances = np.zeros(n_test)
for i in range(n_test):  
    odor_test2 = generate_odorant(50, rdgen_test)
    tag_test2 = project_neural_tag(odor_test2, np.zeros(50), proj_mat)
    # Try the cosine distance, increasing for more similar vectors
    inputs_distances[i] = odor_test.dot(odor_test2) / np.sqrt(np.sum(odor_test**2)) /  np.sqrt(np.sum(odor_test2**2))
    # Rather than just the norm of the difference vector
    #inputs_distances[i] = np.sqrt(np.sum((odor_test - odor_test2)**2))
    tags_distances[i] = jaccard(tag_test, tag_test2)

fig, ax = plt.subplots()
ax.scatter(inputs_distances, tags_distances, s=9)
#ax.set(xlabel=r"$|s_1 - s_2|$", ylabel=r"Jaccard(tag1, tag2)")
ax.set(xlabel=r"$\cos(\theta_{12})$", ylabel=r"Jaccard(tag1, tag2)")
plt.show()
plt.close()

# Functions related to habituation and background fluctuations

In [ ]:
 def time_evolve_habituation_fixed(w_vec0, backgnd, nstep, learnrates):
    """ Take initial conditions for w_i and C_tot, evolve them in time against a fixed odor backgnd. 
    Nothing random here as we don't fluctuate the odor. 
    
    Args:
        w_vec0 (np.ndarray): initial weights 1D vector, should be of same length 
            as vectors in backgnd (number of receptors)
        backgnd (np.ndarray): input vector of the background against which we habituate
        nstep (int): number of time steps to take. 
    
    Returns:
        w_vect (np.ndarray): final weights vector
    """
    # Extract parameters
    if nstep > 1e6:
        raise ValueError("Consider asking for less than 1e6 steps at a time")
    alpha, beta = learnrates
    
    # Initialize variables
    w_vec = w_vec0.copy()
    t = 0  # number of steps taken
    
    # Iterate until satisfing the number of steps asked for
    while t < nstep:
        # Update w, remove the max(s-w, 0) thing for the update rule, see if it still works (it should)
        #x_vec = np.maximum(backgnd - w_vec, 0)
        #w_vec = w_vec + alpha * x_vec - beta* w_vec
        w_vec = w_vec + alpha * backgnd - (alpha + beta)* w_vec
        t += 1

    return w_vec

In [ ]:
def combine_odorants(od1, od2, frac1):
    """ Compute the new input vector after linearly combining two odorant vectors, 
    as frac1*od1 + (1-frac1)*od2
    
    Args:
        od1, od2 (np.ndarrays): input vectors of odorants 1 and 2
        frac1 (float): number between 0 and 1, proportion of od1 in new mixture. 
    Returns:
        od_mix (np.ndarrays): input vector of the mixture.  
    """
    if frac1 < 0.0 or frac1 > 1.0:
        raise ValueError("frac1 should be a float in [0.0, 1.0], not {}".format(frac1))
    return frac1*od1 + (1-frac1)*od2

### Tests

In [ ]:
def one_test_habituation(seed=1484389):
    rdgen_test = np.random.default_rng(seed=seed)
    backgnd_test = generate_odorant(50, rdgen_test)
    w_vec_test = time_evolve_habituation_fixed(np.zeros(50), backgnd_test, 50, (0.05, 0.01))
    
    fig, axes =  plt.subplots(1, 2)
    fig.set_size_inches(5.5, 2.5)
    axes = axes.flatten()
    
    axes[0].bar(range(len(w_vec_test)), w_vec_test, width=.5, color="k")
    axes[0].set(xlabel="PN index i [-]", ylabel=r"$w_i$ [-]")  
    
    # Compare the output tags before and after habituation
    adapt_ratio = 0.05 / (0.05 + 0.01)
    tag_test_initial = project_neural_tag(backgnd_test, np.zeros(50), proj_mat, n_pn_per_kc=6)
    tag_test_final = project_neural_tag(backgnd_test, w_vec_test, proj_mat, n_pn_per_kc=6)
    # print(tag_test_final)  # Should be empty set in noiseless case
    print(tag_test_final)
    tag_vector_final = np.zeros(proj_mat.shape[0])
    tag_vector_final[list(tag_test_final)] = 1.0
    tag_vector_initial = np.zeros(proj_mat.shape[0])
    tag_vector_initial[list(tag_test_initial)] = 1.0
    
    barprops = dict(aspect='auto', interpolation='nearest')
    axes[1].imshow(tag_vector_final.reshape((10, -1)), **barprops, cmap='binary')
    #axes[2].imshow(tag_vector_initial.reshape((1, -1)), **barprops, cmap="gray")
    axes[1].set(xlabel="KC index i [-]", ylabel=r"$z_i$ [-]")
    #axes[1].set_axis_off()
    fig.tight_layout()
    plt.show()
    plt.close()
    
    print("Jaccard initial-final:", jaccard(tag_test_initial, tag_test_final))

In [ ]:
one_test_habituation()

# Supplementary figure 3
Similarity of odor tag B after habituation to A to B's original tag, and its correlation to the similarity between A and B. 

This is the real way to know whether my code reproduces correctly the model. The test is suggested by Shen, Dasgupta, Navlakha, in the section titled "Neural Tags Remain Robust after Habituation to Another Odor".

In [ ]:
# Generate a list of odors
def test_similarity_after_habituation(n_odors=100, n_orn=50, n_kc=2000, n_pn_per_kc=6, 
                    learnrates=(0.05, 0.01), steps=50, metric="jaccard", adapt_kc=True):
    rdgen_test = np.random.default_rng(seed=48225)
    odors_test = [generate_odorant(n_orn, rdgen_test) for i in range(n_odors)]
    proj_test = create_sparse_proj_mat(n_kc=n_kc, n_rec=n_orn, rgen=rdgen_test, fraction_filled=n_pn_per_kc/n_orn)
    # Metric: fraction of original tag still present after habituation to another odor
    # The diagonal should be zero or near, as after self-habituation, 
    # the odor's tag should basically be zero
    tag_simil_matrix = np.zeros([len(odors_test), len(odors_test)])
    tag_simil_before = np.zeros([len(odors_test), len(odors_test)])

    # For all pairs of odors, habituate to one (without noise), then  
    # compute the tag of the other alone before and after habituation
    wzero = np.zeros(n_orn)
    for i in range(len(odors_test)):
        odi = odors_test[i]
        # Habituation to odi: calculate weights w_i after that
        w_vec_afteri = time_evolve_habituation_fixed(wzero, odi, steps, learnrates)
        tag_i_alone = project_neural_tag(odi, wzero, proj_test, adapt_kc=adapt_kc, n_pn_per_kc=n_pn_per_kc)
        for j in range(len(odors_test)):
            # Add odj, compute tag of odj alone
            odj = odors_test[j]
            tag_j_alone = project_neural_tag(odj, wzero, proj_test, adapt_kc=adapt_kc, n_pn_per_kc=n_pn_per_kc)
            tag_dist_before = jaccard(tag_i_alone, tag_j_alone)
            tag_simil_before[i, j] = tag_dist_before
            # After habituation
            tag_j_habit_i = project_neural_tag(odj, w_vec_afteri, proj_test, adapt_kc=adapt_kc, n_pn_per_kc=n_pn_per_kc)
            # Compute similarity of the tags after habituation
            if metric == "jaccard":
                tag_dist = jaccard(tag_j_alone, tag_j_habit_i)
            elif len(tag_j_alone) > 0:
                tag_dist = len(tag_j_alone & tag_j_habit_i) / len(tag_j_alone)
            else:
                tag_dist = 0
            tag_simil_matrix[i, j] = tag_dist
            
    
    return tag_simil_matrix, tag_simil_before

In [ ]:
start_t = perf_counter()
tag_dist_matrix, tag_dist_before = test_similarity_after_habituation(metric="percent", adapt_kc=True)
end_t = perf_counter()
print("Time per pair:", 1000*(end_t - start_t) / tag_dist_matrix.size, "ms")
# Plot statistics and average and median, removing diagonal terms
distrib_tagsim = tag_dist_matrix[~np.eye(tag_dist_matrix.shape[0], dtype=bool)]
distrib_tagbefore = tag_dist_before[~np.eye(tag_dist_before.shape[0], dtype=bool)]

In [ ]:
tag_stats_data = {
    "mean_habit": np.mean(distrib_tagsim), 
    "median_habit": np.median(distrib_tagsim), 
    "mean_ij": np.mean(distrib_tagbefore), 
    "median_ij": np.median(distrib_tagbefore),
    "std_habit": np.std(distrib_tagsim), 
    "std_ij": np.std(distrib_tagbefore)
}
tag_stats_data["pearson"] = (np.mean(distrib_tagbefore*distrib_tagsim) - tag_stats_data["mean_habit"]*tag_stats_data["mean_ij"])/(tag_stats_data["std_habit"]*tag_stats_data["std_ij"])

In [ ]:
tag_stats_data

In [ ]:
# Scatter plot of J[I(s'; s), I(s')] against J[I(s), I(s')]
fig, ax = plt.subplots()
ax.scatter(distrib_tagbefore, distrib_tagsim, s=3)
ax.set(xlabel=r"$J[I(s), I(s')]$", ylabel=r"$J[I(s'; s), I(s')]$")
plt.show()
plt.close()


In [ ]:
fig, axes = plt.subplots(1, 2)
fig.set_size_inches(7, 3)
axes = axes.flatten()
# Pairs of different odorants
axes[0].hist(distrib_tagsim)
axes[0].text(x=0.02, y=0.92, s=r"Habituate to different odorant", 
             transform=axes[0].transAxes)
axes[0].text(x=0.02, y=0.835, s="Mean = {:.4f}".format(np.mean(distrib_tagsim)), 
             transform=axes[0].transAxes)
axes[0].text(x=0.02, y=0.75, s="Median = {:.4f}".format(np.median(distrib_tagsim)), 
             transform=axes[0].transAxes)
axes[0].set(xlabel=r"$J[z_j(0), z_j(t)]$", ylabel="Frequency")

# Pairs of identical odorants
distrib_self_tagsim = tag_dist_matrix[np.eye(tag_dist_matrix.shape[0], dtype=bool)]
axes[1].hist(distrib_self_tagsim, color="xkcd:gold")
axes[1].text(x=0.99, y=0.9, s=r"Habituate to same odorant", 
             transform=axes[1].transAxes, ha="right")
axes[1].text(x=0.99, y=0.8, s="Median = {:.4f}".format(np.median(distrib_self_tagsim)), 
             transform=axes[1].transAxes, ha="right")
axes[1].text(x=0.99, y=0.7, s="Mean = {:.4f}".format(np.mean(distrib_self_tagsim)), 
             transform=axes[1].transAxes, ha="right")
axes[1].set(xlabel=r"$J[z_i(0), z_i(t)]$", ylabel="Frequency")

fig.tight_layout()
plt.show()
plt.close()

# Tag similarity of a mixture after habituation
Figure 4B. 

In [ ]:
def mean_abs_deviation(x):
    return np.mean(np.abs(x - np.mean(x)))

In [ ]:
# Generate a list of odors
def mix_similarity_after_habituation(all_odors=None, n_odors=100, n_orn=50, n_kc=2000, n_pn_per_kc=6, 
        learnrates=(0.05, 0.01), steps=50, metric="jaccard", adapt_kc=True, fix_thresh=None, seed=48225):
    # Initialize projection matrix, odors, containers for scores
    rdgen_mix = np.random.default_rng(seed=seed)
    if all_odors is None:
        all_odors = [generate_odorant(n_orn, rdgen_test) for i in range(n_odors)]
    else:
        n_odors = all_odors.shape[1]
        n_orn = all_odors.shape[0]
        all_odors = all_odors.values.T  # Each row is an odorant now
    proj_mat = create_sparse_proj_mat(n_kc=n_kc, n_rec=n_orn, rgen=rdgen_mix, fraction_filled=n_pn_per_kc/n_orn)
    simil_i_before = np.zeros([len(all_odors), len(all_odors)])  # J(s, s'') before
    simil_j_before = np.zeros([len(all_odors), len(all_odors)])  # J(s', s'') before
    simil_i_after = np.zeros([len(all_odors), len(all_odors)])   # J(s, s'') after
    simil_j_after = np.zeros([len(all_odors), len(all_odors)])   # J(s', s'') after
    
    # For all pairs of odors, habituate to one (without noise), then  
    # compute the tag of the other alone before and after habituation
    sparse_tags = 0
    wzero = np.zeros(n_orn)
    projtag_kwargs = dict(adapt_kc=adapt_kc, n_pn_per_kc=n_pn_per_kc, fix_thresh=fix_thresh)
    for i in range(len(all_odors)):
        odi = all_odors[i]
        tag_i_before = project_neural_tag(odi, wzero, proj_mat, **projtag_kwargs)
        # Habituation to odi: calculate weights w_i after that
        w_vec_afteri = time_evolve_habituation_fixed(wzero, odi, steps, learnrates)
        #tag_i_after = project_neural_tag(odi, w_vec_afteri, proj_mat, **projtag_kwargs)
        for j in range(len(all_odors)):
            if j == i: continue  # Skip odor versus itself
            # Add odj, compute tag of odj beforeand of the mix
            odj = all_odors[j]
            odmix = combine_odorants(odi, odj, 0.8)
            
            # Compute tags of j and mixture before and after
            tag_j_before = project_neural_tag(odj, wzero, proj_mat, **projtag_kwargs)
            tag_mix_before = project_neural_tag(odmix, wzero, proj_mat, **projtag_kwargs)
            #tag_j_after = project_neural_tag(odj, w_vec_afteri, proj_mat, **projtag_kwargs)
            tag_mix_after = project_neural_tag(odmix, w_vec_afteri, proj_mat, **projtag_kwargs)
            if len(tag_mix_after) < 100:
                sparse_tags += 1
            
            # Compute the different tag distances, always using Jaccard
            simil_i_before[i, j] = jaccard(tag_i_before, tag_mix_before)
            simil_j_before[i, j] = jaccard(tag_j_before, tag_mix_before)
            simil_i_after[i, j] = jaccard(tag_i_before, tag_mix_after)
            simil_j_after[i, j] = jaccard(tag_j_before, tag_mix_after)
            
    print("Encountered {} sparse mix tags after habituation; threshold too high?".format(sparse_tags))
    
    return simil_i_before, simil_j_before, simil_i_after, simil_j_after

In [ ]:
start_t = perf_counter()
n_odors = 100
sim_mats = mix_similarity_after_habituation(n_odors=n_odors, n_orn=50, n_kc=2000, n_pn_per_kc=6, 
                    learnrates=(0.05, 0.01), steps=50, metric="jaccard", adapt_kc=True)
end_t = perf_counter()
print("Time per pair:", 1000*(end_t - start_t)/sim_mats[0].size, "ms")

In [ ]:
samples_A_before = sim_mats[0][~np.eye(n_odors).astype(bool)].flatten()
samples_A_after = sim_mats[2][~np.eye(n_odors).astype(bool)].flatten()
samples_B_before = sim_mats[1][~np.eye(n_odors).astype(bool)].flatten()
samples_B_after = sim_mats[3][~np.eye(n_odors).astype(bool)].flatten()

# Prepare two barplots: one for A vs mix before and after, one for B vs mix before and after
fig, axes = plt.subplots(1, 2, sharey=True)
axes = axes.flatten()

barcolors = ["grey", "blue"]
yerr = [mean_abs_deviation(samples_A_before), mean_abs_deviation(samples_A_after)]
axes[0].bar(0, np.median(samples_A_before), yerr=yerr[0], facecolor=barcolors[0], edgecolor="k", alpha=0.5)
axes[0].bar(1, np.mean(samples_A_after), yerr=yerr[1], facecolor=barcolors[1], edgecolor="k", alpha=0.5)
axes[0].scatter(0.075*np.random.normal(size=samples_A_before.size), samples_A_before, s=2,
                color=barcolors[0], alpha=0.7, lw=0.5)
axes[0].scatter(1+0.075*np.random.normal(size=samples_A_after.size), samples_A_after, s=2,
                color=barcolors[1], alpha=0.8, lw=0.5)
axes[0].set_title(r"$s_A(0)$ vs $s_{mix}(t)$")
axes[0].set_ylabel("Jaccard similarity")

yerr = [mean_abs_deviation(samples_B_before), mean_abs_deviation(samples_B_after)]
axes[1].bar(0, np.median(samples_B_before), yerr=yerr[0], facecolor=barcolors[0], edgecolor="k", alpha=0.5)
axes[1].bar(1, np.median(samples_B_after), yerr=yerr[1], facecolor=barcolors[1], edgecolor="k", alpha=0.5)
axes[1].scatter(0.075*np.random.normal(size=samples_B_before.size), samples_B_before, s=2,
                color=barcolors[0], alpha=0.7, lw=0.5)
axes[1].scatter(1+0.075*np.random.normal(size=samples_B_after.size), samples_B_after, s=2,
                color=barcolors[1], alpha=0.8, lw=0.5)
axes[1].set_title(r"$s_B(0)$ vs $s_{mix}(t)$")

for i in range(2):
    axes[i].annotate("Before", xy=(0, axes[i].get_ylim()[1]*0.98), ha="center", va="top", fontsize=8)
    axes[i].annotate("After", xy=(1, axes[i].get_ylim()[1]*0.98), ha="center", va="top", fontsize=8)
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels([r"$t=0$", r"$t=50$"])
    axes[i].set_xlabel("Habituation to A")
    
fig.tight_layout()

plt.show()
plt.close()

## Same as above, but with data
The data is what is actually used in the paper, and seems to give better results (i.e. better Jaccard(B, mix) after habituation)

In [ ]:
set_mean = 10
indata = pd.read_csv('data/hallem2006_TableS1_source.csv', index_col=0, header=0, encoding='utf-8')
normed = indata.T - indata.T.min()
normed = set_mean * normed/normed.mean()
normed.drop(['spontaneous firing rate'], axis=1, inplace=True)
normed

In [ ]:
start_t = perf_counter()
n_odors_data = normed.shape[1]
sim_mats_data = mix_similarity_after_habituation(all_odors=normed, n_odors=n_odors_data, 
                    n_orn=normed.shape[0], n_kc=1000, n_pn_per_kc=3, learnrates=(0.05, 0.01), 
                    steps=50, metric="jaccard", adapt_kc=False, seed=14345)
end_t = perf_counter()
print("Time per pair:", 1000*(end_t - start_t)/sim_mats[0].size, "ms")

In [ ]:
#samples_A_before_data = sim_mats_data[0][~np.eye(n_odors_data).astype(bool)].flatten()
#samples_A_after_data = sim_mats_data[2][~np.eye(n_odors_data).astype(bool)].flatten()
#samples_B_before_data = sim_mats_data[1][~np.eye(n_odors_data).astype(bool)].flatten()
#samples_B_after_data = sim_mats_data[3][~np.eye(n_odors_data).astype(bool)].flatten()
samples_A_before_data = sim_mats_data[0][np.triu_indices(n_odors_data)].flatten()
samples_A_after_data = sim_mats_data[2][np.triu_indices(n_odors_data)].flatten()
samples_B_before_data = sim_mats_data[1][np.triu_indices(n_odors_data)].flatten()
samples_B_after_data = sim_mats_data[3][np.triu_indices(n_odors_data)].flatten()

# Prepare two barplots: one for A vs mix before and after, one for B vs mix before and after
fig, axes = plt.subplots(1, 2, sharey=True)
axes = axes.flatten()

barcolors = ["grey", "blue"]
yerr = [mean_abs_deviation(samples_A_before_data), mean_abs_deviation(samples_A_after_data)]
axes[0].bar(0, np.median(samples_A_before_data), yerr=yerr[0], facecolor=barcolors[0], edgecolor="k", alpha=0.5)
axes[0].bar(1, np.median(samples_A_after_data), yerr=yerr[1], facecolor=barcolors[1], edgecolor="k", alpha=0.5)
axes[0].scatter(0.075*np.random.normal(size=samples_A_before_data.size), samples_A_before_data, s=2,
                color=barcolors[0], alpha=0.7, lw=0.5)
axes[0].scatter(1+0.075*np.random.normal(size=samples_A_after_data.size), samples_A_after_data, s=2,
                color=barcolors[1], alpha=0.8, lw=0.5)
axes[0].set_title(r"$s_A(0)$ vs $s_{mix}(t)$")
axes[0].set_ylabel("Median Jaccard similarity")

yerr = [mean_abs_deviation(samples_B_before_data), mean_abs_deviation(samples_B_after_data)]
axes[1].bar(0, np.median(samples_B_before_data), yerr=yerr[0], facecolor=barcolors[0], edgecolor="k", alpha=0.5)
axes[1].bar(1, np.median(samples_B_after_data), yerr=yerr[1], facecolor=barcolors[1], edgecolor="k", alpha=0.5)
axes[1].scatter(0.075*np.random.normal(size=samples_B_before_data.size), samples_B_before_data, s=2,
                color=barcolors[0], alpha=0.7, lw=0.5)
axes[1].scatter(1+0.075*np.random.normal(size=samples_B_after_data.size), samples_B_after_data, s=2,
                color=barcolors[1], alpha=0.8, lw=0.5)
axes[1].set_title(r"$s_B(0)$ vs $s_{mix}(t)$")

for i in range(2):
    if i == 0:
        ylims = axes[i].get_ylim()
        axes[i].set_ylim(ylims[0], ylims[1]+0.04)
    axes[i].annotate("Before", xy=(0, ylims[1]+0.025), ha="center", va="top", fontsize=10)
    axes[i].annotate("After", xy=(1, ylims[1]+0.025), ha="center", va="top", fontsize=10)
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels([r"$t=0$", r"$t=50$"])
    axes[i].set_xlabel("Habituation to A")
    
fig.tight_layout()


plt.show()
plt.close()

### Nicer graph

In [ ]:
def plot_before_after(samp_before, samp_after, figax=None):
    if figax is None:
        fig, ax = plt.subplots()
        fig.set_size_inches(3, 3)
    else:
        fig, ax = figax

    barcolors = sns.color_palette("mako", n_colors=4)[1:3]
    yerr = [mean_abs_deviation(samp_before), mean_abs_deviation(samp_after)]
    median_before = np.median(samp_before)
    median_after = np.median(samp_after)
    print("Median before:", median_before)
    print("Median_after:", median_after)
    ax.bar(0, median_before, yerr=yerr[0], facecolor=barcolors[0], edgecolor="k", alpha=0.5)
    ax.bar(1, median_after, yerr=yerr[1], facecolor=barcolors[1], edgecolor="k", alpha=0.5)
    ax.scatter(0.075*np.random.normal(size=samp_before.size), samp_before, s=2,
                    color=barcolors[0], alpha=0.7, lw=0.5)
    ax.scatter(1+0.075*np.random.normal(size=samp_after.size), samp_after, s=2,
                    color=barcolors[1], alpha=0.8, lw=0.5)
    ax.set_title(r"$s_A(0)$ vs $s_{mix}(t)$")
    ax.set_ylabel("Jaccard similarity")

    ylims = ax.get_ylim()
    ax.set_ylim(ylims[0], ylims[1]+0.1)
    ax.annotate("Before", xy=(0, ylims[1]+0.05), ha="center", va="top", fontsize=8)
    ax.annotate("After", xy=(1, ylims[1]+0.05), ha="center", va="top", fontsize=8)
    ax.set_xticks([0, 1])
    ax.set_xticklabels([r"$t=0$", r"$t=50$"])
    ax.set_xlabel("Habituation")

    return [fig, ax]

In [ ]:
#samples_A_before_data = sim_mats_data[0][~np.eye(n_odors_data).astype(bool)].flatten()
#samples_A_after_data = sim_mats_data[2][~np.eye(n_odors_data).astype(bool)].flatten()
#samples_B_before_data = sim_mats_data[1][~np.eye(n_odors_data).astype(bool)].flatten()
#samples_B_after_data = sim_mats_data[3][~np.eye(n_odors_data).astype(bool)].flatten()
samples_A_before_data = sim_mats_data[0][np.triu_indices(n_odors_data)].flatten()
samples_A_after_data = sim_mats_data[2][np.triu_indices(n_odors_data)].flatten()
samples_B_before_data = sim_mats_data[1][np.triu_indices(n_odors_data)].flatten()
samples_B_after_data = sim_mats_data[3][np.triu_indices(n_odors_data)].flatten()


fig, ax = plot_before_after(samples_B_before_data, samples_B_after_data)

fig.tight_layout()
# fig.savefig("figures/shen2020_related/habituation_constant_background_data.pdf", bbox_inches="tight", transparent=True)

plt.show()
plt.close()

The rare events depend a lot on the random seed, i.e. the projectino matrix. The mean average deviation thus also depends on it a lot. The median itself, not so much (it oscillates between 0.9607 and 0.9615, depending on the seed. )

In [ ]:
print(np.median(samples_B_after_data) - yerr[1])
print(np.median(samples_B_after_data) )

# Sensitivity to $\tau_0$
Check how the tag sparsifies upon habituation as $\tau_0$ is varied. 

In [ ]:
rdgen_tau = np.random.default_rng(8438341)
proj_mat_tau = create_sparse_proj_mat(2000, 50, rdgen_tau)
backgnd_tau = generate_odorant(50, rdgen_tau)

In [ ]:
w_vec_tau = time_evolve_habituation_fixed(np.zeros(50), backgnd_tau, 50, (0.05, 0.01))

# Compute the output tag with different tau_0 values
thresholds = [22, 20, 15, 10]
tag_initial = project_neural_tag(backgnd_tau, np.zeros(50), proj_mat_tau, fix_thresh=20)
tags = [tag_initial]
for t in thresholds:
    tags.append(project_neural_tag(backgnd_tau, w_vec_tau, proj_mat_tau, fix_thresh=t))

fig, axes = plt.subplots(len(thresholds)+1)
fig.set_size_inches(8, 0.75*(len(thresholds)+1))
print(tag_initial)
for i, tag in enumerate(tags):
    axes[i].eventplot(list(tag), color="k")
    axes[i].set_axis_off()

axes[0].set_title("Initial KC tag", y=0.7)
for i, t in enumerate(thresholds):
    axes[i+1].set_title(r"KC tag for $\tau_0 = {}$".format(t), y=0.75)
fig.tight_layout(h_pad=0.1)
#fig.savefig("figures/shen2020_related/tag_sparsity_depends_on_tau0.pdf", transparent="True", bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
# Performance for discrimination post-habituation for different taus
# Compute the output tag with different tau_0 values
thresholds = [22, 20, 15, 10]
all_sim_mats = []
n_odors = 50
for t in thresholds:
    print("Threshold = ", t)
    sim_mats = mix_similarity_after_habituation(all_odors=None, n_odors=n_odors, 
                    n_orn=50, n_kc=2000, n_pn_per_kc=6, learnrates=(0.05, 0.01), 
                    steps=50, metric="jaccard", adapt_kc=False, fix_thresh=t, seed=14345)
    all_sim_mats.append(sim_mats)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(5, 3)
facecolors = sns.color_palette("mako", len(thresholds))
sparse_mix_tags = [1617, 1175, 49, 0]
for i, t in enumerate(thresholds):
    samples_after = all_sim_mats[i][3][~np.eye(n_odors).astype(bool)].flatten()
    yerr = mean_abs_deviation(samples_after)
    ax.bar(i, np.median(samples_after), yerr=yerr, facecolor=facecolors[i], edgecolor="k", alpha=0.5)
    ax.scatter(i+0.075*np.random.normal(size=samples_after.size), samples_after, s=2,
                    color=facecolors[i], alpha=0.8, lw=0.5)
    ax.annotate("{} mix tags\nwere sparse".format(sparse_mix_tags[i]), xy=(i, 1.2), fontsize=8, ha="center", va="top")
    print(np.median(all_sim_mats[i][2][~np.eye(n_odors).astype(bool)].flatten()))
ax.set_xlabel(r"$\tau_0$")
ax.set_xticks(range(len(thresholds)))
ax.set_xticklabels(thresholds)
ax.set_ylabel(r"Jaccard($s_A(0), s_{mix}(t)$)")
fig.tight_layout()
#fig.savefig("figures/shen2020_related/habituation_constant_background_effect_tau0.pdf", transparent=True)
plt.show()
plt.close()